Implementation of policy iteration (Sutton and Barto, section 4.3, page 80)

In [267]:
import numpy as np

In [274]:
class Agent:

    '''
    Initializes the agent
    @param rows - The number of rows in the grid world
    @param columns - The number of columns in the grid world
    @param terminal_state - The final state the agent is trying to find the best path to
    '''
    def __init__(self, rows, columns, terminal_state, theta=0.05, gamma=0.85):
        # self.grid = np.random.rand(rows, columns) * -1 # Initializes
        self.rows = rows
        self.columns = columns
        self.values = np.zeros((rows, columns)) # Initializes all state values to zero
        self.policies = self.initializePolicies()
        # self.policies = self.initializePolicies() # Initializes policies for each cell,
        #                                           # policies[row][column] is an array where the value
        #                                           # at index 0 is the probability that up is the optimal choice,
        #                                           # 1 is right, 2 is down, 3 is left (clockwise around Cartesian)
        self.terminal_state = terminal_state # Where the end state is located on the grid
        self.theta = theta # A value close to zero signifying completion, determines the accuracy of the policy estimation
        self.gamma = gamma # A value signifying by how much to discount future rewards 

    '''
    Initializes the policies for each cell in the grid world
    '''
    def initializePolicies(self):
        policies = np.empty((self.rows, self.columns, 4))

        # Fill the array with random values that sum to 1
        for i in range(self.rows):
            for j in range(self.columns):
                policies[i, j] = np.random.dirichlet(np.ones(4))
                # policies[i, j] = np.zeros(4)
                # policies[i, j][np.random.randint(4)] = 1

        return policies

    def isValidState(self, coords):
        return coords[0] >= 0 and coords[0] < self.rows and coords[1] >= 0 and coords[1] < self.columns

    '''
    Gets the four available successor states
    Note: some of the returned successor states may be invalid, a validity check is needed
    @param coords - coordinates in tuple form (x, y) of our state
    @return a list of successors
    '''
    def getAvailableSuccessorStates(self, coords):
        successors = []

        successors.append((coords[0] - 1, coords[1]))
        successors.append((coords[0], coords[1] + 1))
        successors.append((coords[0] + 1, coords[1]))
        successors.append((coords[0], coords[1] - 1))

        return successors

    '''
    Get the discounted value of each successor state
    @return a numpy array of successor state values
    '''
    def getDiscountedValuesForSuccessors(self, state):
        values = np.zeros(4)
        
        successors = self.getAvailableSuccessorStates(state) # Get possible next states
        
        value = 0
        for successor in range(len(successors)):
            if self.isValidState(successors[successor]):
                probability_successor_chosen = self.policies[state][successor]
                successor_value = self.getReward(successors[successor]) + self.gamma * self.values[successors[successor]]
                values[successor] = probability_successor_chosen * successor_value

        return values

    '''
    Update the policy based on the values of successor states
    '''
    def updatePolicy(self, coords):
        successors = self.getAvailableSuccessorStates(coords)
        values = np.ones(4) / 4
        for successor in range(len(successors)):
            if self.isValidState(successors[successor]):
                values[successor] = self.values[successors[successor][0]][successors[successor][1]] + self.getReward(successors[successor])
            else:
                values[successor] = 0
        
        sum = np.sum(values)
        policies = np.ones(4) / 4 if sum == 0 else values / sum

        self.policies[coords[0]][coords[1]] = policies

    '''
    Gets the reward of a particular state
    High reward if terminal state, no reward otherwise
    '''
    def getReward(self, state):
        if state[0] == self.terminal_state[0] and state[1] == self.terminal_state[1]: # Terminal state
            return 100
        return 0
        

    '''
    Performs a policy evaluation step
    Update our value function based on our current policy
    '''
    def policyEvaluation(self):
        while True:
            delta = 0 # This is the check to know when to stop evaluation
            for coords, old_value in np.ndenumerate(self.values): # Iterate over states - in the form ((row, column), value)
                
                value = np.sum(self.getDiscountedValuesForSuccessors(coords)) # Cell value is equal to the sum of the discounted successor state values
                self.values[coords] = value

                delta = max(delta, np.absolute(old_value - value)) # Get the change in the value

            if delta < self.theta: # Check if the update was significant
                break
    
    '''
    Performs a policy improvement step
    Generate a new, improved policy based on our new value function
    '''
    def policyImprovement(self):
        while True:
            stable = True
            for coords, value in np.ndenumerate(self.values): # Iterate over all states
                old_action = np.argmax(self.policies[coords]) # Get the previous policy value
                self.updatePolicy(coords) # Update the policy based on the updated value function
                action = np.argmax(self.policies[coords]) # Get the best action according to our new policy
                if old_action != action: # If our policy has changed, it is unstable, so we perform a policy evaluation step
                    stable = False
                    
            if not stable:
                self.policyEvaluation()
            else:
                break
    
    '''
    Outputs the policy in a readable format
    '''
    def outputPolicy(self):

        mapping = {0: "Up", 1: "Right", 2: "Down", 3: "Left"}

        output = []

        for row in range(len(self.policies)):
            row_output = []
            for cell in range(len(self.policies[row])):
                if row == self.terminal_state[0] and cell == self.terminal_state[1]:
                    print('{:^6s}'.format("End"), end="")
                else:
                    print('{:^6s}'.format(mapping[np.argmax(self.policies[row][cell])]), end="")
            print()

    def valueUpdate(self, state):
        values = np.zeros(4)
        
        successors = self.getAvailableSuccessorStates(state) # Get possible next states
        
        value = 0
        for successor in range(len(successors)):
            if self.isValidState(successors[successor]):
                probability_successor_chosen = self.policies[state[0]][state[1]][successor]
                successor_value = self.getReward(successors[successor]) + self.gamma * self.values[successors[successor]]
                values[successor] = probability_successor_chosen * successor_value

        return values

    '''
    Value iteration alternative to policy iteration
    '''
    def valueIteration(self):
        while True:
            delta = 0
            for coords, old_value in np.ndenumerate(self.values): # Iterate over each cell
                value = np.amax(self.getDiscountedValuesForSuccessors(coords)) # Take the maximum discounted value from the successor states
                self.values[coords] = value # Update current state value
                delta = max(delta, np.absolute(old_value - value))

            if delta < self.theta: # If our policy is stable
                break   
        
        for coords, old_value in np.ndenumerate(self.values): # Update the policy at each set of coordinates
            self.updatePolicy(coords)


In [275]:
a = Agent(10, 10, (2, 4))

In [276]:
a.valueIteration()

In [277]:
a.outputPolicy()

Right Right  Down Right Right  Down  Left  Left  Left  Left 
Right  Down  Down  Down  Down  Down  Left  Left  Left  Left 
Right Right Right Right  End   Left  Left  Left  Left  Left 
Right   Up    Up    Up    Up    Up   Left  Left  Left  Left 
Right Right   Up    Up    Up    Up    Up    Up    Up   Left 
  Up    Up    Up  Right   Up   Left   Up    Up    Up    Up  
  Up  Right   Up  Right   Up   Left  Left   Up   Left  Left 
Right Right Right Right   Up   Left  Left   Up   Left  Left 
Right Right Right Right   Up    Up   Left  Left  Left  Left 
Right Right   Up    Up    Up    Up    Up    Up    Up   Left 


In [278]:
a.policyImprovement()

In [279]:
a.values.round(1)

array([[ 0. ,  0.1,  0.4,  0.3,  4.8, 22.4,  1.3,  0.1,  0. ,  0. ],
       [ 0.1,  0.2,  1.4,  3.6, 19.2, 46.5, 11.7,  0.5,  0. ,  0. ],
       [ 0. ,  1.9,  5.8, 30.9, 34.1, 78. ,  4.4,  1. ,  0.2,  0. ],
       [ 0.1,  0.4,  1. ,  4.6, 10.2,  7.8,  6.2,  0.9,  0.2,  0. ],
       [ 0. ,  0.2,  0.5,  0.5,  0.8,  0.5,  0.3,  0.1,  0.1,  0. ],
       [ 0. ,  0. ,  0.1,  0. ,  0.6,  0.1,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.4,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.2,  0.1,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [280]:
a.outputPolicy()

Right Right  Down Right Right  Down  Left  Left  Left  Left 
Right  Down  Down  Down  Down  Down  Left  Left  Left  Left 
Right Right Right Right  End   Left  Left  Left  Left  Left 
Right   Up    Up    Up    Up    Up   Left  Left  Left  Left 
Right Right   Up    Up    Up    Up    Up    Up    Up   Left 
  Up    Up    Up  Right   Up   Left   Up    Up    Up    Up  
  Up  Right   Up  Right   Up   Left  Left   Up   Left  Left 
Right Right Right Right   Up   Left  Left   Up   Left  Left 
Right Right Right Right   Up    Up   Left  Left  Left  Left 
Right Right   Up    Up    Up    Up    Up    Up    Up   Left 
